In [ ]:
import toml

import numpy as np
import matplotlib.pyplot as plt

import pylinac
import imageio

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys

from pymedphys._experimental.wlutz import reporting

from pymedphys._experimental.wlutz import main as _wlutz
from pymedphys._experimental.wlutz import imginterp as _imginterp, utilities as _utilities, pylinacwrapper as _pylinac_wrapper

In [ ]:
data_files = pymedphys.zip_data_paths('previously_failing_iview_images.zip')
data_files

In [ ]:
collimator_angles = toml.load([item for item in data_files if item.suffix == '.toml'][0])
collimator_angles

In [ ]:
jpg_paths = {
    item.name: item for item in data_files if item.suffix == '.jpg'
}
jpg_paths

In [ ]:
edge_lengths = [20, 26]
penumbra = 2
bb_diameter = 8

algorithms = [
    "PyMedPhys",
    f"PyLinac v{pylinac.__version__}"
]

In [ ]:
# results = {}

# for filename, full_image_path in jpg_paths.items():
#     x, y, image = _wlutz.load_iview_image(full_image_path)
#     icom_field_rotation = -collimator_angles[filename]

#     results[filename] = {}
#     for algorithm in algorithms:
#         field_centre, bb_centre = _wlutz.calculate(
#             full_image_path,
#             algorithm,
#             bb_diameter,
#             edge_lengths,
#             penumbra,
#             icom_field_rotation,
#         )
        
#         results[filename][algorithm] = {
#             'field_centre': field_centre,
#             'bb_centre': bb_centre
#         }

In [ ]:
# results

In [ ]:
# algorithm = algorithms[1]
filename = "000058A7.jpg"
full_image_path = jpg_paths[filename]
icom_field_rotation = -collimator_angles[filename]
icom_field_rotation += 3.5

x, y, image = _wlutz.load_iview_image(full_image_path)
field = _imginterp.create_interpolated_field(x, y, image)
straightened_field = _utilities.create_centralised_field(
    field, [0,0], icom_field_rotation
)

# x = np.arange(-20,20,0.25)
# y = x
xx, yy = np.meshgrid(x, y)

straightened_image = straightened_field(xx, yy)

In [ ]:
plt.figure(figsize=(10,10))
plt.pcolormesh(xx, yy, straightened_image)
plt.axis('equal')
# plt.xlim([-20,20])
# plt.ylim([-20,20])

In [ ]:
# imageio.imsave("pylinac_offset.png", straightened_image)

In [ ]:
field_centre, bb_centre = _pylinac_wrapper.run_wlutz_raw(x, y, straightened_image, fill_errors_with_nan=True, pylinac_version="2.2.6")

In [ ]:
field_centre

In [ ]:
bb_centre

In [ ]:
fig, axs = reporting.image_analysis_figure(
    x,
    y,
    straightened_image,
    bb_centre,
    field_centre,
    0,
    bb_diameter,
    edge_lengths,
    penumbra,
)

In [ ]:
algorithm = algorithms[0]
# filename = "000057E2.jpg"
# full_image_path = jpg_paths[filename]

for filename, full_image_path in jpg_paths.items():

    x, y, image = _wlutz.load_iview_image(full_image_path)
    icom_field_rotation = -collimator_angles[filename]

    field_centre, bb_centre = _wlutz.calculate(
        full_image_path,
        algorithm,
        bb_diameter,
        edge_lengths,
        penumbra,
        icom_field_rotation,
    )

    fig, axs = reporting.image_analysis_figure(
        x,
        y,
        image,
        bb_centre,
        field_centre,
        icom_field_rotation,
        bb_diameter,
        edge_lengths,
        penumbra,
    )

    axs[0, 0].set_title(algorithm)

    print(filename)
    print(field_centre, bb_centre)
    
    plt.show()

    